在构建prompt时, 可以通过构建一个**少量示例列表**去进一步格式化prompt, 这是一种简单但强大的指导生成的方式, 在某些情况下可以**显著提高模型性能**。

少量示例提示模板可以由**一组示例**或一个负责从定义的集合中选择**一部分示例**的**示例选择器**构建。

* 前者: 使用 **FewShotPromptTemplate** 或 **FewShotChatMessagePromptTemplate**
* 后者: 使用 **Example selectors (示例选择器)**

每个示例的结构都是一个**字典**, 其中**键**是输入变量, **值**是输入变量的值。

# 少量示例的提示词模板的使用

FewShotPromptTemplate: 与PromptTemplate一起使用

FewShotChatMessagePromptTemplate: 与ChatPromptTemplate一起使用

Example selectors(示例选择器):

## 1、FewShotPromptTemplate的使用

未提供示例的情况

In [1]:
from langchain_openai import ChatOpenAI

import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

res = chat_model.invoke("2 🦜 9 是多少?")
print(res.content)

2 🦜 9 是一个表达式，可能是用来表示某种运算或关系，但在数学上并没有明确的含义。如果你能提供更多的上下文或解释这个符号的意思，我将更好地帮助你。


举例2: 使用FewShotPromptTemplate

In [4]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# 创建PromptTemplate的实例
example_prompt = PromptTemplate.from_template(
    template="input:{input}\noutput:{output}",
)

# 提供一些示例
examples = [
    {"input": "北京天气怎么样", "output": "北京市"},
    {"input": "南京下雨吗", "output": "南京市"},
    {"input": "武汉热吗", "output": "武汉市"}
]

# 创建FewShotPromptTemplate的实例
few_shot_temp = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="input:{input}\noutput:", #声明在示例后面的提示词模板
    input_variables=["input"],
)

few_shot_temp.invoke({"input": "天津会下雨吗?"})

StringPromptValue(text='input:北京天气怎么样\noutput:北京市\n\ninput:南京下雨吗\noutput:南京市\n\ninput:武汉热吗\noutput:武汉市\n\ninput:天津会下雨吗?\noutput:')

调用大模型之后：

In [6]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


chat_model = ChatOpenAI(model="gpt-4o-mini")

chat_model.invoke(few_shot_temp.invoke({"input": "天津会下雨吗?"}))

AIMessage(content='天津市', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 50, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-Ce7Glun67Ob6KGanzeLYKvANdYj4d', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c75ec6cf-947a-4aff-8faa-693d12709ec0-0', usage_metadata={'input_tokens': 50, 'output_tokens': 2, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 2、FewShotChatMessagePromptTemplate的使

举例1:

In [1]:
from langchain.prompts import (FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
) #
# 1.示例消息格式
examples = [
{"input": "1+1等于几？", "output": "1+1等于2"},
{"input": "法国的首都是？", "output": "巴黎"}
] #
# 2.定义示例的消息格式提示词模版
msg_example_prompt = ChatPromptTemplate.from_messages([
("human", "{input}"),
("ai", "{output}"),
])
# 3.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=msg_example_prompt,
    examples=examples
) #
# 4.输出格式化后的消息
print(few_shot_prompt.format())

Human: 1+1等于几？
AI: 1+1等于2
Human: 法国的首都是？
AI: 巴黎


In [4]:
# 1.导入相关包
from langchain_core.prompts import (FewShotChatMessagePromptTemplate,
ChatPromptTemplate)
# 2.定义示例组
examples = [
    {"input": "2🦜2", "output": "4"},
    {"input": "2🦜3", "output": "8"},
] #
# 3.定义示例的消息格式提示词模版
example_prompt = ChatPromptTemplate.from_messages([
    ('human', '{input} 是多少?'),
    ('ai', '{output}')
])
# 4.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples, # 示例组
    example_prompt=example_prompt, # 示例提示词词模版
) #
print(few_shot_prompt)
# 5.输出完整提示词的消息模版
final_prompt = ChatPromptTemplate.from_messages(
    [('system', '你是一个数学奇才'),
    few_shot_prompt,
    ('human', '{input}'),]
) #6.
# 提供大模型
import os
import dotenv
from langchain_openai import ChatOpenAI
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(model="gpt-4o-mini",
temperature=0.4)
chat_model.invoke(final_prompt.invoke(input="2🦜4")).content

examples=[{'input': '2🦜2', 'output': '4'}, {'input': '2🦜3', 'output': '8'}] input_variables=[] input_types={} partial_variables={} example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input} 是多少?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})])


'16'

## 3、Example selectors (示例选择器)
举例1:

In [5]:
# 1.导入相关包
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
import os
import dotenv
from langchain_openai import OpenAIEmbeddings
dotenv.load_dotenv()
# 2.定义嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-ada-002"
) #
# 3.定义示例组
examples = [
    {
        "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵?",
        "answer": """
        接下来还需要问什么问题吗？
        追问：穆罕默德·阿里去世时多大年纪？
        中间答案：穆罕默德·阿里去世时享年74岁。
        """,
    },
    {
        "question": "craigslist的创始人是什么时候出生的？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：谁是craigslist的创始人？
        中级答案：Craigslist是由克雷格·纽马克创立的。
        """,
    },
    {
        "question": "谁是乔治·华盛顿的外祖父？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：谁是乔治·华盛顿的母亲？
        中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。
        """,
    },
    {
        "question": "《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：《大白鲨》的导演是谁？
        中级答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
        """,
    },
] #
# 4.定义示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 这是可供选择的示例列表
    examples,
    # 这是用于生成嵌入的嵌入类，用于衡量语义相似性
    embeddings_model,
    # 这是用于存储嵌入并进行相似性搜索的 VectorStore 类
    Chroma,
    # 这是要生成的示例数量
    k=1,
) #

question = "玛丽·鲍尔·华盛顿的父亲是谁?"
selected_examples = example_selector.select_examples({"question": question})
print(f"与输入最相似的示例：{selected_examples}")
# for example in selected_examples:
# print("\n")
# for k, v in example.items():
# print(f"{k}: {v}")

与输入最相似的示例：[{'answer': '\n        接下来还需要问什么问题吗？\n        追问：谁是乔治·华盛顿的母亲？\n        中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。\n        ', 'question': '谁是乔治·华盛顿的外祖父？'}]


In [7]:
# 1.导入相关包
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings
# 2.定义示例提示词模版
example_prompt = PromptTemplate.from_template(
template="Input: {input}\nOutput: {output}",
)
# 3.创建一个示例提示词模版
examples = [
    {"input": "高兴", "output": "悲伤"},
    {"input": "高", "output": "矮"},
    {"input": "长", "output": "短"},
    {"input": "精力充沛", "output": "无精打采"},
    {"input": "阳光", "output": "阴暗"},
    {"input": "粗糙", "output": "光滑"},
    {"input": "干燥", "output": "潮湿"},
    {"input": "富裕", "output": "贫穷"},
] #
# 4.定义嵌入模型
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
# 5.创建语义相似性示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=2,
) #或者 #examp

# le_selector = SemanticSimilarityExampleSelector(
#     examples,
#     embeddings,
#     FAISS,
#     k=2
# ) #6.
# 定义小样本提示词模版
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个词组的反义词",
    suffix="Input: {word}\nOutput:",
    input_variables=["word"],
)
response = similar_prompt.invoke({"word":"忧郁"})
print(response.text)

给出每个词组的反义词

Input: 高兴
Output: 悲伤

Input: 阳光
Output: 阴暗

Input: 忧郁
Output:


举例1: 从物理磁盘的yaml文件中读取prompt数据

In [9]:
from langchain_core.prompts import load_prompt
from dotenv import load_dotenv

load_dotenv()

prompt = load_prompt("prompt.yaml", encoding="utf-8")
# print(prompt)
print(prompt.format(name="年轻人", what="滑稽"))

请给年轻人讲一个关于滑稽的故事


从物理磁盘的json文件中读取prompt

In [10]:
from langchain_core.prompts import load_prompt
from dotenv import load_dotenv

load_dotenv()

prompt = load_prompt("prompt.json", encoding="utf-8")
# print(prompt)
print(prompt.format(name="年轻人", what="滑稽"))

请给年轻人讲一个关于滑稽的故事。
